In [ ]:
%pip install python-dotenv asyncio_throttle xmltodict aiohttp

In [ ]:
import aiohttp
import asyncio
from asyncio_throttle import Throttler
import pandas as pd
import os
import urllib.parse
import xmltodict
import requests

In [ ]:
%load_ext dotenv
%dotenv keys.env

In [ ]:
api_key = os.getenv("bibKey")
headers = {
    "Authorization": f"apikey {api_key}",
    "Accept": "application/json",
    'limit' : '100',
}
base = "https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/"

In [ ]:
course_key = os.getenv("courseKey")

course_headers = {
    "Authorization": f"apikey {course_key}",
    "Accept": "application/json",
}

In [ ]:
def build_db():
  analytics_key= os.getenv("analyticsKey")
  path_encoded = "%2Fshared%2FMiraCosta%20College%2001CACCL_MIRACOSTA%2Fcitation%20ids"
  path = urllib.parse.unquote(path_encoded)
  analytics_url = 'https://api-na.hosted.exlibrisgroup.com/almaws/v1/analytics/reports'

  analytics_headers = {
    "path" : path,
    "apiKey": analytics_key,
    'limit' : 1000
  }
  y=0
  flag = 'false'

  try:
    while y < 1:
  # Make the initial request
      analytics_response = requests.get(analytics_url, params=analytics_headers)
      data_dict = xmltodict.parse(analytics_response.content)
      db_list.append([row for row in data_dict['report']['QueryResult']['ResultXml']['rowset']['Row']])
      
  # Exchange the report path for redemption token. Pass on subsequent runs.
      try:
        del analytics_headers["path"]
        analytics_headers['token'] = data_dict['report']['QueryResult']['ResumptionToken']   
      except KeyError:
        pass

  # Build dictionary of column names to map onto dataframe.
      if flag == 'false':
        for i in data_dict['report']['QueryResult']['ResultXml']['rowset']['xsd:schema']['xsd:complexType']['xsd:sequence']['xsd:element']:
          column_number = i['@name'] 
          column_name = i['@saw-sql:columnHeading']
          columns[column_number] = column_name
        flag = 'true'

  # Break out of the loop if the 'IsFinished' element returns 'true'

      if data_dict['report']['QueryResult']['IsFinished'] == 'true':
        y+=1
        break

  except KeyError as error:
    print(f'Key Error: {error}')

In [ ]:
def shape_db(db_list):

# Separate out rows containing data (type: dict) 
# from empty rows (type: str)

  cleaned_db = []

  for i in db_list:
    for j in i:  
      if type(j) == dict:
        cleaned_db.append(j)
        continue
      else:
        pass

  return cleaned_db

In [ ]:
# Function builds out the dictionary that becomes a row in the final dataframe. Contains physical item information.
def get_phys_item(
    i,
):  
    b = {
        "title": i["bib_data"]["title"],
        "Barcode": i["item_data"]["barcode"],
        "call number": i["holding_data"]["call_number"],
        "permanent location": i["item_data"]["location"]["desc"],
        "edition": i["bib_data"]["complete_edition"],
    }
    
    try:
        b['isbn'] = i["bib_data"]["isbn"]
    except KeyError:
        b['isbn'] = 'none'
    try:
        b['author'] = i["bib_data"]["author"]
    except KeyError:
        b['author'] = 'none'
    try:
        b["due back date"] = i["holding_data"]["due_back_date"]
    except KeyError:
        b["due back date"] = "none"
    try:
        b["in temp location"] = i["holding_data"]["in_temp_location"]
    except KeyError:
        b["in temp location"] = "none"
    try:
        b["temp location"] = i["holding_data"]["temp_location"]["value"]
    except KeyError:
        b["temp location"] = "none"
    try:
        b["temp policy"] = i["holding_data"]["temp_policy"]["value"]
    except KeyError:
        b["temp policy"] = "none"
    try:
        b["temp call number"] = i["holding_data"]["temp_call_number"]
    except KeyError:
        b["temp call number"] = "none"

    return b

In [ ]:
# Creates the URL endpoints needed to make citation requests.
def build_url(df):  

    a = []

    baseURL = "https://api-na.hosted.exlibrisgroup.com/almaws/v1/courses/"

    for index, row in df.iterrows():
        course_id, citation_id, reading_list_id = (
            row["Course ID"],
            row["Citation Id"],
            row["Reading List Id"],
        )

        url = (
            baseURL
            + course_id
            + "/reading-lists/"
            + reading_list_id
            + "/citations/"
            + citation_id
        )

        a.append(url)

    return a

In [ ]:
# Async handler for making bibs (phys item) api calls. Works together with get_bibs()
async def get_bibs_list(
    df, headers
):  
    throttler = Throttler(rate_limit=25)
    async with aiohttp.ClientSession() as client:

        awaitables = [
            get_bibs(client=client, throttler=throttler, headers=headers, mms=i)
            for i in df["MMS Id"]
        ]

        itemList = await asyncio.gather(*awaitables)

    return itemList

In [ ]:
# Works with get_bibs_list() to batch collection item information.
async def get_bibs(
    client, throttler, headers, mms
):  
    items = []
    async with throttler:
        try:
            resp = None
            url = base + mms + "/holdings/ALL/items"
            async with client.get(url, headers=headers) as session:
                if session.status != 200:
                    resp = await session.text()
                    session.raise_for_status()
                resp = await session.json()
                if (
                    resp["total_record_count"] == 0
                ):  # Checks to see how many items in the record. If none, adds the MMS ID of the record to the list so we can make citation calls on those MMS Ids
                    items.append(mms)
                else:
                    items.append(resp)
        except Exception as e:
            print(f"{e} for {mms}")

    return items

In [ ]:
# Used with get_ebook_list() to make batch citation calls to get ebook information.
async def get_ebook(
    client, throttler, headers, mms, df
):  
    ebooks = []
    async with throttler:
        try:
            resp = None
            courseInfo = df.loc[
                df["MMS Id"] == mms
            ]  # Uses the MMS ID to pull the table row, then uses values from that row to apply course information (code, instructor, end date)
            url = str(courseInfo.url.values[0])
            async with client.get(url, headers=headers) as session:
                if session.status != 200:
                    resp = await session.text()
                    session.raise_for_status()
                resp = await session.json()
                b = {
                    "title": resp["metadata"][
                        "title"
                    ],  # Build out the citation information as a dictionary, appends it into the ebooks list.
                    "Barcode": "none - ebook",
                    "call number": "none",
                    "permanent location": "none",
                    "due back date": "none",
                    "in temp location": "none",
                    "temp location": "none",
                    "temp policy": "none",
                    "temp call number": "none",
                    "course code": courseInfo["Course Code"].values[0],
                    "course end date": courseInfo["Current Course End Date"].values[0],
                    "instructor": courseInfo["Course Instructor"].values[0],
                }

                try:
                    b['isbn'] = resp["metadata"]["isbn"]
                except KeyError:
                    b['isbn'] = 'none'
                try:
                    b['edition'] = resp["metadata"]["edition"]
                except KeyError:
                    b['edition'] = 'none'
                try:
                    b['author'] = resp["metadata"]["author"]
                except KeyError:
                    b['author'] = 'none'
                ebooks.append(b)
        except Exception as e:
            print(f"{e} for {mms}")

    return ebooks

In [ ]:
# Works with get_ebook() to make batch courses calls to retrieve citation information. Uses course_headers dict and course key.
async def get_ebook_list(
    elec_items, course_headers
):  
    throttler = Throttler(rate_limit=25)
    async with aiohttp.ClientSession() as client:

        awaitables = [
            get_ebook(
                client=client, throttler=throttler, headers=course_headers, mms=i, df=df
            )
            for i in elec_items
        ]

        itemList = await asyncio.gather(*awaitables)

    return itemList

In [ ]:
# Simple string formatter to remove special characters from the MMS ids.
def formatter(
    str,
):  
    str1 = str.replace("[", "")
    str2 = str1.replace("]", "")
    str3 = str2.replace("'", "")

    return str3

In [ ]:
# Build out each row for a physical item. Uses MMS ID to find the appropriate row 
# containing static course information (instructor, code, end date) and applies that to each item.
def get_physical_citations(
    p_list, df, cit_list
):  
    for i in p_list:
        mms = i[0]["item"][0]["bib_data"]["mms_id"]
        courseInfo = df[df["MMS Id"].str.contains(mms)]    # return rows where mms matches
        
        for j in i[0]["item"]: 
            for index, row in courseInfo.iterrows():
              a = get_phys_item(j)
              a["course code"] = row["Course Code"]
              a["course end date"] = row["Current Course End Date"]
              a["instructor"] = row["Course Instructor"]
        
              cit_list.append(a)

In [ ]:
# Separates out the MMS IDs for citation calls from the item records of physical item calls.
def build_lists(
    itemlist, phys_items, elec_items
):  
    for i in itemlist:
        try:
            if i[0]["total_record_count"] > 0:
                phys_items.append(i)
        except TypeError as error:
            elec_items.append(i)
        except IndexError:
            pass

In [ ]:
# Extracts out the returned citation records from the async ebooks function, then loads those into the final list.
def load_ebooks(
    ebooks, citationItems
):  
    for i in ebooks:
        for j in i:
            citationItems.append(j)

In [ ]:
# Strip the Z from the due back date being passed from Alma.
def strip_date(a):
    try:
        return a.strip("Z")
    except AttributeError:
        pass

In [ ]:
# Initialize empty lists we'll use.
phys_items, temp_elec_items, citationItems, elec_items, db_list = (
    [] for i in range(5)
) 

columns = {}

In [ ]:
# Retrieve report data
build_db()

In [ ]:
print(len(db_list))

In [ ]:
# Remove rows containing no data
db_rows = shape_db(
    db_list
)

In [ ]:
# Create dataframe from report data
df = pd.DataFrame(
    db_rows
)

In [ ]:
# Map report column names onto dataframe
df = df.rename(
    columns=columns
)

In [ ]:
# Build out our endpoint urls for making course calls (for citation information we use for ebooks)
df["url"] = build_url(
    df
)  

In [ ]:
# Make bibs calls to get physical item information.
itemlist = await get_bibs_list(
    df, headers
)  

In [ ]:
# Separates out physical item information (phys_items), and MMS ids to pull citation information (temp_elec_items)
build_lists(
    itemlist, phys_items, temp_elec_items
)  

In [ ]:
# format the mms IDs
e = [formatter(str(i)) for i in temp_elec_items]  

In [ ]:
# Remove duplicate MMS id's
elec_items = [*set(e)]  

In [ ]:
# Make course api calls to retrieve citation information
ebooks = await get_ebook_list(
    elec_items, course_headers
)  

In [ ]:
# Extract out citation information and adds it to the final list.
load_ebooks(
    ebooks, citationItems
)  

In [ ]:
# Extract relevent physical item information from the records we received, then adds them to the citationItems list.
get_physical_citations(
    phys_items, df, citationItems
)  

In [ ]:
# Generate a dataframe from the collection of physical / electronic item dictionaries.
c = pd.DataFrame.from_records(
    citationItems
)  

In [ ]:
# Drop duplicate rows while keeping the last instance.
d = c.drop_duplicates(
    subset = ['course code', 'Barcode'],
    keep= 'last').reset_index(drop=True) 

In [ ]:
# Format date columns to match
d['due back date'] = [strip_date(s) for s in d['due back date']]
d['course end date'] = pd.to_datetime(d['course end date'], infer_datetime_format=True)
d['course end date'] = d['course end date'].dt.date

In [ ]:
# Choose the columns we want in the final report, and export it as xlsx.
columns = [
    "course code",
    "instructor",
    "title",
    "author",
    "edition",
    "call number",
    "isbn",
    "Barcode",  
    "permanent location",
    "temp location",
    "in temp location",
    "temp call number",
    "temp policy",
    "due back date",
    "course end date",
]
d = d[columns]
d = d.sort_values(by=["course code"])
d.to_excel("cts.xlsx", index=False)